In [1]:
import json
import requests
import pandas as pd
import datetime
from bs4 import BeautifulSoup
from google.colab import  drive

drive.mount('drive')


Mounted at drive


In [3]:
r = requests.get('https://tw.stock.yahoo.com/fund/offshore')
soup = BeautifulSoup(r.text,'html.parser')
divs = soup.find_all(class_="Fxg(1) Fxs(1)")

coms = {}

for div in divs :
  if div.find("a") is not None :
    id = div.find("a")['href']
    id = id[id.rfind('/')+1:]
    coms[id] = div.text
    
coms


{'0C00009DF1': '富蘭克林證券投資顧問股份有限公司',
 '0C00009DF3': '法銀巴黎證券投資顧問股份有限公司',
 '0C00001OKT': '野村證券投資信託股份有限公司',
 '0C00001RZU': '鋒裕匯理證券投資信託股份有限公司',
 '0C00001RZT': '聯博證券投資信託股份有限公司',
 '0C000023XG': '宏利證券投資信託股份有限公司',
 '0C00001OU1': '摩根證券投資信託股份有限公司',
 '0C00001VY7': '新光證券投資信託股份有限公司',
 '0C00001QS5': '瀚亞證券投資信託股份有限公司',
 '0C00001TIR': '富達證券投資信託股份有限公司',
 '0C00009DEV': '富盛證券投資顧問股份有限公司',
 '0C00009UCU': '中國信託證券投資信託股份有限公司',
 '0C0000BUKK': '安睿宏觀證券投資顧問股份有限公司',
 '0C00001TIQ': '德銀遠東證券投資信託股份有限公司',
 '0C000015CC': '第一金證券投資信託股份有限公司',
 '0C00001TA7': '柏瑞證券投資信託股份有限公司',
 '0C00008U6S': '合作金庫證券投資信託股份有限公司',
 '0C00009DEW': '康和証券投資顧問股份有限公司',
 '0C00009DF0': '霸菱證券投資顧問股份有限公司',
 '0C00001KL2': '安聯證券投資信託股份有限公司',
 '0C00001NPU': '富邦證券投資信託股份有限公司',
 '0C00001OD6': '匯豐中華證券投資信託股份有限公司',
 '0C00001OOS': '景順證券投資信託股份有限公司',
 '0C00001QP1': '貝萊德證券投資信託股份有限公司',
 '0C00001QRO': '保德信證券投資信託股份有限公司',
 '0C00001SUA': '品浩太平洋證券投資顧問股份有限公司',
 '0C00001TAB': '台中銀證券投資信託股份有限公司',
 '0C00001TIT': '瑞銀證券投資信託股份有限公司',
 '0C00003GSA': '施羅德證券投資信託股份有限公司',
 '0C00009DEQ': '中租證

In [4]:
dirn = "drive/MyDrive/export/" + datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=8))).strftime("%Y%m%d_%H%M%S")
!mkdir $dirn

for com in coms :
  print("Loading : " + coms[com])
  offset = '0'
  l = []
  while(offset is not None) :
    r = requests.get('https://tw.stock.yahoo.com/_td/api/resource/FundServices.fundsSearch;query={"offset":"%s","onshore":0,"providerId":"%s"}' % (offset,com))
    tmpj = json.loads(r.text)
    l += tmpj['funds']
    print(offset + " => " +tmpj['pagination']['nextOffset'] if tmpj['pagination']['nextOffset'] is not None else "End")
    offset = tmpj['pagination']['nextOffset']
  data = { "id" : [], "基金名稱" : [], "幣別" : [], "更新日期" : [], "淨值" : [], "1W" : [], "1M" : [], "3M" : [], "6M" : [], "1Y" : [], "3Y" : [], "5Y" : [], "YTD" : [] }
  performances = {"1W" : False,"1M" : False,"3M" : False,"6M" : False,"1Y" : False,"3Y" : False,"5Y" : False,"YTD" : False}
  for row in l :
    data["id"].append(row["id"])
    data["基金名稱"].append(row["name"])
    data["幣別"].append(row["currencyName"])
    data["更新日期"].append(row["latestPriceDate"])
    data["淨值"].append(row["price"])

    for col in row["performance"]["returns"] :
      data[col["period"]].append(col["rate"])
      performances[col["period"]] = True
    for ps in performances :
      if performances[ps] == True :
        performances[ps] = False
      else :
        data[ps].append("NONE")

  df = pd.DataFrame.from_dict(data)
  fname = coms[com] + '.csv';
  df.to_csv(fname, index = False)
  !cp $fname $dirn

Loading : 富蘭克林證券投資顧問股份有限公司


KeyError: ignored